## How to create a closed-loop optimisation using a pool-based sampler as the benchmark

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.optimisation import Optimisation

In [ ]:
# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0)
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2])

In [ ]:
# Create the objective objects
obj1 = RegressionObjective(name="objective1", obj_max_bool=True, lower_bound=0.0, upper_bound=100.0)
obj2 = RegressionObjective(name="objective2", obj_max_bool=False, lower_bound=0.0, upper_bound=100.0)
obj_list = ObjectivesList([obj1, obj2])

### Setting up the sample pool

To benchmark using an existing collection of samples, the `PoolBased` class can be used as the sampler. Note that to use this method of benchmarking, a benchmark function object is not needed.

In [ ]:
from nemo_bo.opt.samplers import PoolBased

# Instantiate the sampler to create the sample pool
# X_pool and Y_pool arrays represent hypothetical datasets that are in the pool of samples to select from during the optimisation
sampler = PoolBased(X_pool, Y_pool)

Pool-based sampling requires the use of an expected improvement-type algorithm (`ExpectedImprovement` in NEMO). So the `NSGAImprovement` acquisition function in NEMO (based on the U-NSGA-III algorithm) cannot be used. This is because this type of optimisation requires selecting candidates from a user-defined pool of samples, which goes against how NSGA-type algorithms work, since they require generation of their own samples *in situ*.

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler)

### Starting the optimisation run
Start the optimisation using the convenient run function that will run for the specified number of iterations. Ensure that `number_of_iterations * num_candidates` in the acquisition function does exceed the number of samples in the pool.

When the plot_progress keyword argument is True, a pareto plot will be created at every iteration.

In [ ]:
# X and Y arrays represent a hypothetical initial dataset
optimisation_data = optimisation.run(X, Y, number_of_iterations=50, plot_progress=True)